UC3MAL201 Machine Learning Session 11 Tutorial 11 Leon Eriksen Helgeland -
First written 09.09.2020

# Neural Network - with Keras

## 1. Dataset description

- Name of the datset: MNIST
- Contents:     28x28 pixel grayscale images
- Images contain: Handwritten single digits, 0-9
- Total dataset length: 70,000 images. 60,000 train-set and 10,000 test-set 
- Total dataset row length: 785 columns.
- First column: 1 class column, digit 0-9.
- All other columns : 784 feature columns, pixels from 0 to 255

## 2. Data pre-processing

In [ ]:
%%time
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [ ]:
%%time
train = pd.read_csv(r'C:\Users\leon.helgeland\OneDrive\S05-MAL-2020\S05MAL - Session 11 - Neural Network with Keras - Live Demo\Dataset for the tutorial-20200908\mnist_train.csv',header=None)
test = pd.read_csv(r'C:\Users\leon.helgeland\OneDrive\S05-MAL-2020\S05MAL - Session 11 - Neural Network with Keras - Live Demo\Dataset for the tutorial-20200908\mnist_test.csv',header=None)
X_test = test.iloc[:,1:].apply(lambda a: a*(0.99/255)+0.01) # Rescaling 0-255 pixel values to 0.01-1
X_train = train.iloc[:,1:].apply(lambda a: a*(0.99/255)+0.01) # Rescaling 0-255 pixel values to 0.01-1
# Above: This rescaling process helps to prevent having 0 values as inputs which can cause weight update problems during network synthesis.

enc = OneHotEncoder(categories="auto", sparse=False, dtype=np.int16) # Removed: handle_unknown='ignore'
y_train = enc.fit_transform(train.iloc[:,:1].to_numpy().reshape(-1, 1)) # OneHotEncoding single column to binary columns
y_test = enc.fit_transform(test.iloc[:,:1].to_numpy().reshape(-1, 1)) # OneHotEncoding single column to binary columns

## 3. Modelling process

In [ ]:
%%time
def model_fit(X_train, y_train, config):
    n_input = config[0]
    n_output = config[1]
    n_nodes = config[2]
    epoch = config[3]
    batch = config[4]
    # Define and Compile
    model = Sequential() # Model activation
    model.add(Dense(n_nodes, input_dim=n_input, activation='relu')) # Input layer with 'relu' function
    # Test adding these layers:
    # - Conv + Pool x2
    # - Conv x2 (optional)
    # - Conv + Pool x1 (optional)
    # - Full x2
    model.add(Dense(n_output, activation='softmax')) # Output layer with 'softmax' function 
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) # Compling the output 
    # Fit the model
    model.fit(X_train, y_train, epochs=epoch, batch_size=batch)
    return model

## 4. Prediction process

# Testing

In [ ]:
df = pd.read_csv('dataset/')
def rescale_image(a)
    '''
    Takes a image value between 0-255 as input and converts it to a value between 0.01 and 1.
    '''
    return lambda a : a*(0,99/255)+0.01
df[:-1] = [[rescale_image(pixel) for pixel in row] for row in df[:-1]]

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import KFold
import tensorflow.keras.optimizers
import os
import tensorflow
print(tensorflow.__version__)

In [ ]:
%%time
dataset_train_file = 'mnist_train.csv'
dataset_test_file = 'mnist_test.csv'
dataset_directory = r'C:\Users\leon.helgeland\OneDrive\S05-MAL-2020\S05MAL - Session 11 - Neural Network with Keras - Live Demo\Dataset for the tutorial-20200908'
#'~/GitHub/BigData/dataset11'
# import train and test dataset
train = pd.read_csv(f'{dataset_directory}/{dataset_train_file}',header=None)
test = pd.read_csv(f'{dataset_directory}/{dataset_test_file}',header=None)
# Import dataset
X_train = train.iloc[:,1:].apply(lambda a: a*(0.99/255)+0.01)
y_train = train.iloc[:,:1]
X_test = test.iloc[:,1:].apply(lambda a: a*(0.99/255)+0.01)
y_test = test.iloc[:,:1]
# The class labels are integer values, encoding them into one-hot representation https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(categories="auto", sparse=False, dtype=np.int16) # handle_unknown='ignore'
y_train = enc.fit_transform(y_train.to_numpy().reshape(-1, 1))
y_test = enc.fit_transform(y_test.to_numpy().reshape(-1, 1))

## Dataset preparation
Rescale the data to values between 0.01 and 1, by multiplying each pixel value in the dataset by (0.99/255) and adding 0.01 to the result to get a new pixel value.

In [ ]:
def model_fit(X_train, y_train, config):
    n_input = config[0]
    n_output = config[1]
    n_nodes = config[2]
    epoch = config[3]
    batch = config[4]
    # Define and Compile
    model = Sequential()
    model.add(Dense(n_nodes, input_dim=n_input, activation='relu'))
    model.add(Dense(n_output, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # Fit the model
    model.fit(X_train, y_train, epochs=epoch, batch_size=batch)
    return model

# predict with the fit model
def model_predict(model, testdata):
    Prediction = model.predict(testdata, verbose=0)
    return Prediction

# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

In [ ]:
inputs = 784 # Input features
outputs = 10 # Output features
nodes = [5,10,15,20,35,50] # Test amount of nodes in layer

configs = []
for n in nodes:
    config_ = [inputs, outputs, n, 75, 250]
    configs.append(config_)

In [ ]:
# Kfolding
config_lossess = []
accuracy_per_config = []

kf = KFold(n_splits=5, shuffle=True, random_state=42)

counter = 0
for config in configs:
    counter+=1
    fold_count = 0

    fold_losses = []
    fold_accuracy = []
    
    for fold_train, fold_test in kf.split(X_train, y_train):
        fold_count+=1
        nn_model = model_fit(X_train[fold_train], y_train[fold_train], config)
        scores = neural_network_model.evaluate(X_train[fold_test], y_train[fold_test], verbose=0)
        print(f"Score for config {counter} fold {fold_count}: {neural_network_model.metrics_names[0]}={scores[0]}; {neural_network_model.metrics_names[1]}={scores[1]*100}%")
        
        fold_losses.append(scores[0])
        fold_accuracy.append(scores[1]*100)

    config_lossess.append(fold_losses)
    accuracy_per_config.append(fold_accuracy)

In [ ]:
print("Score per config: ")
for j in range(0, len(accuracy_per_config)):
    print('^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^')
    print(f'Score for config {j+1}. Nodes:{configs[j][2]}')
    print('Average scores for all folds:')
    print(f'> Loss: {np.mean(config_lossess[j])}')
    print(f'> Accuracy: {np.mean(accuracy_per_config[j])} (+- {np.std(accuracy_per_config[j])})')
    print('')

In [ ]:
configy = [784, 10, 20, 10, 1]
neural_network = model_fit(X_train, y_train, configy)

In [ ]:
# Evaluate the model
prediction = model_predict(neural_network, X_test)

PredClass = list()
ActualClass = list()
for row in range(len(y_test)):
    index_maxscore1 = max(range(len(y_test[row])), key=y_test[row].__getitem__)
    ActualClass.append(index_maxscore1)
    index_maxscore2 = max(range(len(prediction[row])), key=prediction[row].__getitem__)
    PredClass.append(index_maxscore2)
    print('Expected=%d, Got=%d' % (index_maxscore1, index_maxscore2))


accuracy = accuracy_metric(PredClass, ActualClass)
accuracy

In [ ]:
    import tensorflow as tf
    if tf.test.gpu_device_name():
        print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
    else:
        print("Please install GPU version of TF")